<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/%E6%AD%A3%E5%90%A6%E5%88%A4%E5%AE%9A_%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#正否判定_解説
# %%
import pandas as pd

# %%
df = (
    pd
    .read_csv("../data_raw/①Y2_SSW対象_宛名番号ハッシュ化.csv")
    # .drop(columns="宛名番号")
)
df
# %%
score = (
    pd
    .read_excel("../doc/20240116_和泉市身体的虐待スコアリスト.xlsx")
)
score
# %%
score = (
    df
    .merge(
        score
        [["宛名番号", "セグメントID", "決定木_被虐待確率"]]
        .rename(columns={"宛名番号": "HS_宛名番号"})
        .drop_duplicates()
        , on="HS_宛名番号"
        , how="inner"
    )#df データフレームと、加工した score データフレームを「HS_宛名番号」で内部結合します（how="inner"）。内部結合により、両方のデータフレームに存在する「HS_宛名番号」に一致する行だけが結果に含まれます。
    # .to_csv("../doc/①Y2_SSW対象_宛名番号ハッシュ化+決定木セグメント.csv", index=False)
)
score
# %%
res = (
    pd
    .read_excel("../data_raw/【ハッシュ化】対象者抽出 (1).xlsx")
)
res
# %%
(
    score
    .drop_duplicates()#重複行の削除:
    .merge(res, how="inner")#score データフレームと res データフレームを内部結合
    .pivot_table(
        index=["SSW対象", "相談回数", "セグメントID"],
        columns="SSWアセスメント結果",
        values="HS_宛名番号",
        aggfunc="nunique"#集計関数として nunique を使用します。nunique はユニークな値の数をカウントする関数です。
    )
    .fillna(0)
    .assign(TTL = lambda df: df.sum(axis=1) - df["未"])#各行の合計値 (df.sum(axis=1)) から「未」列の値を引いた結果を、新しい列 TTL に格納します。
    .T#データフレームを転置します
    .assign(TTL = lambda df: df.sum(axis=1))
    .T
    .assign(O率 = lambda df: df["〇"] / df["TTL"])#「〇」列の値を TTL 列の値で割った割合を計算し、新しい列 O率 に格納します。この列は「〇」の割合を示します。
)
# %%
mrt_nri = (
    res
    .rename(columns={"HS_宛名番号": "宛名番号"})
    .drop_duplicates()
    .merge(ptest.drop_duplicates())#res と ptest データフレームを結合します。重複行が削除された後に結合されるので、結合結果には重複がありません。
    .loc[lambda df: df["SSWアセスメント結果"].isin({"〇", "×"})]#SSWアセスメント結果 列が「〇」または「×」の行のみをフィルタリングします。
    # ["宛名番号"].nunique()
    [[
        "宛名番号",
        "SSWアセスメント結果",
        "1.要対協_要対協_被虐待数_昨年度累計_昨年差分",
        "5.家児相.csv_家児相_ケース区分=虐待_昨年度累計回数",
        "3.生活保護.csv_被保護世帯員個人情報一覧_世帯被生保延べ人数",
        "3.ひとり親医療受給者.csv_ひとり親医療受給者_受給回数_昨年差分",
        "1.要対協_要対協_被虐待数_昨年度累計",
        "5.家児相.csv_家児相_ケース区分=相談_昨年度累計回数",
        "2.出生～４か月健診.csv_出生～４か月健診_総合判定=要観察_フラグ",
        "3.こども医療受給者.csv_こども医療受給者_受給回数",
        "4.世帯人数.csv_世帯人数_3歳未満人数",
    ]]#これれだけ抽出
    .drop_duplicates()#最終的に重複行が削除された結果が mrt_nri に格納されます。
)
# %%
mrt_nri.to_excel("../doc/20240305_NRI事務局検証用マート.xlsx")

# %%
(
    mrt_nri
    .assign(**{"1.要対協_要対協_被虐待数_昨年度累計_昨年差分": lambda df: df['1.要対協_要対協_被虐待数_昨年度累計_昨年差分'] <= 0.5})
    .assign(**{"5.家児相.csv_家児相_ケース区分=虐待_昨年度累計回数": lambda df: df['5.家児相.csv_家児相_ケース区分=虐待_昨年度累計回数'] <= 0.5})
    .assign(**{"3.生活保護.csv_被保護世帯員個人情報一覧_世帯被生保延べ人数": lambda df: df['3.生活保護.csv_被保護世帯員個人情報一覧_世帯被生保延べ人数'] <= 1.5})
    .assign(**{"3.ひとり親医療受給者.csv_ひとり親医療受給者_受給回数_昨年差分": lambda df: df['3.ひとり親医療受給者.csv_ひとり親医療受給者_受給回数_昨年差分'] <= 0.5})
    .assign(**{"1.要対協_要対協_被虐待数_昨年度累計": lambda df: df['1.要対協_要対協_被虐待数_昨年度累計'] <= 0.5})
    .assign(**{"5.家児相.csv_家児相_ケース区分=相談_昨年度累計回数": lambda df: df['5.家児相.csv_家児相_ケース区分=相談_昨年度累計回数'] <= 0.5})
    .assign(**{"2.出生～４か月健診.csv_出生～４か月健診_総合判定=要観察_フラグ": lambda df: df['2.出生～４か月健診.csv_出生～４か月健診_総合判定=要観察_フラグ'] <= 0.5})
    .assign(**{"3.こども医療受給者.csv_こども医療受給者_受給回数": lambda df: df['3.こども医療受給者.csv_こども医療受給者_受給回数'] <= 0.5})
    .assign(**{"4.世帯人数.csv_世帯人数_3歳未満人数": lambda df: df['4.世帯人数.csv_世帯人数_3歳未満人数'] <= 0.5})
    .groupby("SSWアセスメント結果")
    .sum()
    .T
)

# %%
'1.要対協_要対協_被虐待数_昨年度累計_昨年差分 <= 0.5'
'5.家児相.csv_家児相_ケース区分=虐待_昨年度累計回数 <= 0.5'
'3.生活保護.csv_被保護世帯員個人情報一覧_世帯被生保延べ人数 <= 1.5'
'3.ひとり親医療受給者.csv_ひとり親医療受給者_受給回数_昨年差分 <= 0.5'
'1.要対協_要対協_被虐待数_昨年度累計 <= 0.5'
'5.家児相.csv_家児相_ケース区分=相談_昨年度累計回数 <= 0.5'
'2.出生～４か月健診.csv_出生～４か月健診_総合判定=要観察_フラグ <= 0.5'
'3.こども医療受給者.csv_こども医療受給者_受給回数 <= 0.5'
'4.世帯人数.csv_世帯人数_3歳未満人数 <= 0.5'
